In [1]:
import keras
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Input
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import concatenate, Lambda, Reshape
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam
import keras.backend as K
from keras.datasets import cifar10
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.applications.inception_v3 import InceptionV3

from sklearn.metrics import classification_report, confusion_matrix
import cv2

import itertools
import numpy as np
import os
import copy
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import SVG

In [3]:
def getimages(direct):
    im=[]
    label=[]
    lab=0
    for image in os.listdir(direct):
        lab = int(image.split('P')[0])
        lab=lab-1
        imag=cv2.imread(direct+r'/'+image)
        imag=imag[55:550,100:690]
        imag=cv2.resize(imag,(200,200))
        im.append(imag)
        label.append(lab)
    #Suffle all images
    return shuffle(im, label)

#Obtain both train and test images
im,label=getimages("D:\Images_dataset_MTI")

#Convert to numpy array
Images=np.array(im)
Labels=np.array(label)
x1, xtest, y1, ytest = train_test_split(Images, Labels, random_state=0)
xtrain, xvalidation, ytrain, yvalidation=train_test_split(x1, y1, random_state=0)

In [4]:
ytrain0 = keras.utils.to_categorical(ytrain, 6)
yvalidation0 = keras.utils.to_categorical(yvalidation, 6)
ytest0=keras.utils.to_categorical(ytest, 6)

In [23]:
# Small VGG-like model
def simpleVGG(im_Input, num_classes, name="vgg"):
    name = [name+str(i) for i in range(12)]
    
    # convolution and max pooling layers
    vgg = Conv2D(32, (3, 3), padding='same', activation='relu', name=name[0])(im_Input)
    vgg = Conv2D(32, (3, 3), padding='same', activation='relu', name=name[1])(vgg)
    vgg = MaxPooling2D(pool_size=(2,2), name=name[2])(vgg)
    vgg = Dropout(0.25, name=name[3])(vgg)
    vgg = Conv2D(64, (3, 3), padding='same', activation='relu', name=name[4])(vgg)
    vgg = Conv2D(64, (3, 3), padding='same', activation='relu', name=name[5])(vgg)
    vgg = MaxPooling2D(pool_size=(2,2), name=name[6])(vgg)
    vgg = Dropout(0.25, name=name[7])(vgg)

    # classification layers
    vgg = Flatten(name=name[8])(vgg)
    vgg = Dense(512, activation='relu', name=name[9])(vgg)
    vgg = Dropout(0.5, name=name[10])(vgg)
    vgg = Dense(num_classes, activation='softmax', name=name[11])(vgg)
    return vgg

In [24]:
# Large VGG-like model
def fatVGG(input_im, num_classes, name="vgg"):
    name = [name+str(i) for i in range(17)]
    
    # convolution and max pooling layers
    vgg = Conv2D(32, (3, 3), padding='same', activation='relu', name=name[0])(input_im)
    vgg = Conv2D(32, (3, 3), padding='same', activation='relu', name=name[1])(vgg)
    vgg = MaxPooling2D(pool_size=(2,2), name=name[2])(vgg)
    vgg = Dropout(0.25, name=name[3])(vgg)
    vgg = Conv2D(64, (3, 3), padding='same', activation='relu', name=name[4])(vgg)
    vgg = Conv2D(64, (3, 3), padding='same', activation='relu', name=name[5])(vgg)
    vgg = MaxPooling2D(pool_size=(2,2), name=name[6])(vgg)
    vgg = Dropout(0.25, name=name[7])(vgg)
    vgg = Conv2D(128, (3, 3), padding='same', activation='relu', name=name[8])(vgg)
    vgg = Conv2D(128, (3, 3), padding='same', activation='relu', name=name[9])(vgg)
    vgg = Conv2D(128, (3, 3), padding='same', activation='relu', name=name[10])(vgg)
    vgg = MaxPooling2D(pool_size=(2,2), name=name[11])(vgg)
    vgg = Dropout(0.25, name=name[12])(vgg)

    # classification layers
    vgg = Flatten(name=name[13])(vgg)
    vgg = Dense(512, activation='relu', name=name[14])(vgg)
    vgg = Dropout(0.5, name=name[15])(vgg)
    vgg = Dense(num_classes, activation='softmax', name=name[16])(vgg)
    return vgg

In [25]:
im_Input = Input(shape=(xtrain.shape[1:]), name="input")
baseVGG = fatVGG(im_Input, 6, "base")
baseModel = Model(im_Input, baseVGG)
# compile
baseModel.compile(loss='categorical_crossentropy',
                   optimizer=Adam(),
                   metrics=['accuracy'])

In [28]:
# Number of training/testing examples per batch
batch_size = 50

# Training epochs
epochs = 10

# train
history=baseModel.fit(xtrain, ytrain0,
               batch_size=batch_size,
               epochs=epochs,
               validation_data=(xvalidation, yvalidation0))

Epoch 1/10
20/20 [==============================] - 95s 5s/step - loss: 0.2792 - accuracy: 0.8875 - val_loss: 0.2504 - val_accuracy: 0.9027
Epoch 2/10
20/20 [==============================] - 105s 5s/step - loss: 0.2317 - accuracy: 0.9068 - val_loss: 0.2470 - val_accuracy: 0.8906
Epoch 3/10
20/20 [==============================] - 109s 5s/step - loss: 0.2443 - accuracy: 0.9068 - val_loss: 0.2415 - val_accuracy: 0.8997
Epoch 4/10
20/20 [==============================] - 108s 5s/step - loss: 0.2070 - accuracy: 0.9159 - val_loss: 0.2278 - val_accuracy: 0.9149
Epoch 5/10
20/20 [==============================] - 106s 5s/step - loss: 0.2077 - accuracy: 0.9058 - val_loss: 0.2258 - val_accuracy: 0.9179
Epoch 6/10
10/20 [==============>...............] - ETA: 50s - loss: 0.2268 - accuracy: 0.9040

KeyboardInterrupt: 

In [29]:
from sklearn.metrics import accuracy_score
ypred=baseModel.predict(xtest)
print("Test:",accuracy_score(ytest, np.argmax(ypred, axis=1)))

Test: 0.9179954441913439


In [17]:
np.argmax(ypred, axis=1)

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,

In [19]:
ytest

array([1, 1, 0, 4, 3, 5, 4, 4, 5, 4, 1, 0, 0, 5, 3, 3, 3, 0, 5, 1, 0, 3,
       4, 2, 2, 3, 2, 4, 1, 2, 2, 2, 5, 3, 5, 5, 0, 5, 4, 1, 3, 5, 0, 4,
       5, 3, 5, 4, 1, 1, 2, 3, 0, 4, 5, 2, 2, 1, 1, 0, 0, 5, 2, 4, 0, 1,
       2, 0, 5, 3, 4, 4, 1, 2, 0, 1, 5, 4, 2, 1, 4, 5, 2, 2, 3, 2, 0, 0,
       4, 1, 0, 0, 1, 3, 0, 2, 2, 2, 0, 2, 3, 0, 2, 3, 5, 2, 4, 1, 5, 5,
       4, 1, 0, 0, 3, 1, 3, 5, 1, 2, 3, 5, 3, 3, 4, 4, 2, 1, 4, 0, 4, 5,
       4, 0, 2, 3, 2, 2, 4, 3, 5, 1, 4, 2, 4, 0, 0, 4, 1, 0, 0, 0, 3, 1,
       2, 5, 5, 1, 1, 0, 2, 3, 4, 3, 2, 0, 0, 5, 4, 1, 0, 2, 2, 2, 1, 1,
       1, 3, 0, 5, 5, 3, 0, 5, 3, 3, 5, 2, 2, 3, 4, 4, 0, 0, 4, 1, 3, 0,
       3, 2, 4, 2, 3, 5, 0, 0, 3, 0, 4, 4, 4, 3, 1, 5, 4, 0, 1, 3, 5, 1,
       4, 4, 0, 3, 2, 0, 3, 3, 0, 2, 0, 1, 2, 1, 2, 3, 0, 1, 1, 1, 1, 0,
       2, 1, 0, 1, 5, 0, 2, 4, 3, 3, 2, 4, 5, 2, 0, 1, 1, 3, 1, 0, 2, 0,
       0, 2, 3, 2, 5, 2, 5, 3, 2, 1, 2, 4, 3, 1, 3, 2, 4, 4, 1, 5, 1, 4,
       4, 1, 0, 4, 0, 2, 3, 5, 0, 5, 0, 2, 3, 4, 0,